In [1]:
# Input: 3D tensor [len_sequence len_minibatch dim_vector]
# here, each input is a sequence without minibatch: [1,1,3]
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Sets the seed for generating random numbers. And returns a torch._C.Generator object.
torch.manual_seed(1)

In [12]:
lstm = nn.LSTM(3, 3)  # Input dim_feat is 3, dim_output is 3
inputs = [autograd.Variable(torch.randn((1, 3)))
          for _ in range(5)]  # length_sequence = 5,dim_feat = 3

# initialize the hidden state h_t tuble 1x2
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn((1, 1, 3))))

for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension

# concatenate inputs 
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
# clearn out hidden states 
hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(
    torch.randn((1, 1, 3))))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out) # sequence of output 5x3
print(hidden) # 2 1x1x3 variable 

Variable containing:
(0 ,.,.) = 
 -0.2054 -0.1134  0.0751

(1 ,.,.) = 
 -0.2836 -0.1156 -0.0135

(2 ,.,.) = 
 -0.1793 -0.2396 -0.0653

(3 ,.,.) = 
 -0.2135 -0.1629 -0.0275

(4 ,.,.) = 
 -0.2167 -0.0932 -0.1355
[torch.FloatTensor of size 5x1x3]

(Variable containing:
(0 ,.,.) = 
 -0.2167 -0.0932 -0.1355
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.8033 -0.3279 -0.2088
[torch.FloatTensor of size 1x1x3]
)


In [4]:
# example of LSTM for Part-of-speech Tagging 
# input: a sentence of words: w1,w2,...,wM
# tag set: T; tag of word wi: yi; prediction of tag: ŷi
# so the structure prediction is a sequence:ŷ1,...,ŷM
# hidden state at time step i: hi
# assign each tag a index, just like word_to_ix

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

# [sentence, tah], what's meaning of these tags 
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6


{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [6]:
# LSTM model for tagging sentence words 
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores
        

In [8]:
# train the model 
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss() # negative log loss 
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        
        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()
        
        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Variables of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
        
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
# See what the scores are after training
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)

# The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#  for word i. The predicted tag is the maximum scoring tag.
# Here, we can see the predicted sequence below is 0 1 2 0 1
# since 0 is index of the maximum value of row 1,
# 1 is the index of maximum value of row 2, etc.
# Which is DET NOUN VERB DET NOUN, the correct sequence!
print(tag_scores) # output of a sentence 


Variable containing:
-0.7895 -1.2153 -1.3891
-0.8144 -1.2048 -1.3574
-0.7875 -1.1315 -1.5029
-0.7773 -1.2257 -1.3992
-0.8012 -1.1161 -1.4976
[torch.FloatTensor of size 5x3]

Variable containing:
-0.0980 -2.9951 -3.1390
-4.7820 -0.0125 -5.5056
-1.5610 -4.2193 -0.2544
-0.0642 -4.0497 -3.1060
-3.4617 -0.0324 -7.5341
[torch.FloatTensor of size 5x3]



In [ ]:
# word embeddings with a representatio derived from the characters of the word 
# let c_w be the character-level representation fo word w
# x_w is the word embedding as before
# the input of LSTM is a concatenation: x_w + c_w
# To do a sequence model over characters, you will have to embed characters. 
# The character embeddings will be the input to the character LSTM.
